In [ ]:
import pandas as pd
import numpy as np
import re
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chart = pd.read_pickle('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 6 - 데이터 완성/chart.pickle')
chart['가사'] = chart['가사'].str.replace('\n',' ')
chart.head()

,연도,제목,가수,가사,발매년도,장르,좋아요 수
0,2023,I Don't Think That I Like Her,Charlie Puth,Get her name and get her number Find out all o...,2022,POP,"214,990"
1,2023,Dangerously,Charlie Puth,This is gonna hurt but I blame myself first ca...,2016,POP,"217,897"
2,2023,STAY,"The Kid LAROI, Justin Bieber",I do the same thing I told you that I never wo...,2021,POP,"293,599"
3,2023,That's Hilarious,Charlie Puth,Look how all the tables Look how all the table...,2022,POP,"184,630"
4,2023,"Steal The Show (From ""Elemental"")",Lauv,Started out on a one-way train Always knew whe...,2023,"POP, 애니메이션/웹툰, 키즈, 만화","185,174"


In [ ]:
genre = pd.read_pickle('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 6 - 데이터 완성/genre.pickle')
genre['가사'] = genre['가사'].str.replace('\n',' ')
genre.head()

,장르,제목,가수,가사,발매년도,좋아요 수
0,댄스,비행기,거북이,Yes Turtles Forth album New mind New song 철없을 ...,2006,"112,502"
1,댄스,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),아이유,하얀 눈이 내려올 때면 온 세상이 물들을 때면 눈꽃이 피어나 또 빛이 나 눈이 부신...,2010,"174,230"
2,댄스,좋은 날,아이유,어쩜 이렇게 하늘은 더 파란 건지 오늘따라 왜 바람은 또 완벽한지 그냥 모르는 척 ...,2010,"154,444"
3,댄스,I Don't Care,2NE1,hey playboy it's about time and your time's up...,2009,"79,758"
4,댄스,너랑 나,아이유,시곌 보며 속삭이는 비밀들 간절한 내 맘속 이야기 지금 내 모습을 해쳐도 좋아 나를...,2011,"155,098"


In [ ]:
playlist = pd.read_pickle('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 6 - 데이터 완성/playlist.pickle')
playlist['가사'] = playlist['가사'].str.replace('\n',' ')
playlist.head()

,플리,제목,가수,가사,장르,발매일,좋아요
0,melody_1,you should see me in a crown,Billie Eilish,Bite my tongue bide my time Wearing a warning ...,POP,2018,"28,392"
1,melody_1,Dead Meat,Sean Lennon,Dead meatdon't you know you're dead meatyou ju...,POP,2006,324
2,melody_1,Look What You Made Me Do,Taylor Swift,I don't like your little games Don't like your...,POP,2017,"65,931"
3,melody_1,MAD,Caravan Palace,"I turn on the engine, better be on clock I bum...",POP,2023,77
4,melody_1,Killer,Valerie Broussard,Standing on the highway with my thumb out Ever...,POP,2019,"16,078"


In [ ]:
print("chart :", chart.shape[0], "\ngenre :", genre.shape[0], "\nplaylist :", playlist.shape[0])

chart : 6414 
genre : 2826 
playlist : 32600


### Upstage

In [ ]:
! pip install -qU langchain langchain-upstage langchain_community python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.19.1 which is incompatible.


In [ ]:
# # @title set API key
# import os
# import getpass

# from pprint import pprint

# import warnings

# warnings.filterwarnings("ignore")

# if "google.colab" in str(get_ipython()):
#     # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
#     from google.colab import userdata

#     os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
# else:
#     # Running locally. Please set the UPSTAGE_API_KEY in the .env file
#     from dotenv import load_dotenv

#     load_dotenv()

# if "UPSTAGE_API_KEY" not in os.environ:
#     os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [ ]:
form_fields = [
    {
        "1": ["keyword1", "keyword2", "keyword3"],
        # "Description": "Find three Korean noun keywords that can summarize the lyrics, in Korean."
        "2": ["emotion1", "emotion2", "emotion3"]
        # "Description": "Provide three noun emotions that can describe the lyrics, in Korean (e.g., 행복, 슬픔, 우울, 설렘, )"
    }
]

In [ ]:
my_info = chart['가사'][157]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

form_filling_prompt_teample = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """You are a form filler. Fill out the form based on the lyrics below.

            **Instructions:**
            - Extract **three keywords** and **three emotions** from the given lyrics.
            - Find three Korean noun keywords that can summarize the lyrics, in Korean.
            - Provide three noun emotions that can describe the lyrics, in Korean (e.g. 행복, 슬픔, 우울, 설렘)
            - Make sure the answers are in **Korean**. If some words are english or numbers, **translate them into Korean.**
            - Make sure the answers are all **nouns**.
            - Provide the output in the following **format**, **without any additional discription**:
            - **Make sure you follow the given format**
              {{
                "1" : ["keyword1", "keyword2", "keyword3"],
                "2" : ["emotion1", "emotion2", "emotion3"]
              }}
            - Do your best to extract the information.
            - If you can guess from the lyrics, fill it out and make "[GUESS]".
            - If unsure, fill the missing part with "모르겠습니다."

            **Lyrics:**
            {my_info}

            **Form to Fill:**
            {form_fields}

            **Expected Output Example:**
            {{
              "1" : ["하늘", "고백", "사랑"],
              "2" : ["설렘", "행복", "포근함"]
            }}
            """,
        )
    ]
)

In [ ]:
# write prompt
prompt = form_filling_prompt_teample.format_messages(
    my_info=my_info,
    form_fields=form_fields,
)

# print(prompt[0].content)

In [ ]:
solar_mini = ChatUpstage(model="solar-1-mini-chat")
chain_mini = form_filling_prompt_teample | solar_mini | StrOutputParser()

In [ ]:
filled_form = chain_mini.invoke(
    {
        "my_info": my_info,
        "form_fields": form_fields,
    }
)

print(filled_form)

AuthenticationError: Error code: 401 - {'error': {'message': 'API key suspended due to insufficient credit. Register your payment method at https://console.upstage.ai/billing to continue.', 'type': 'invalid_request_error', 'param': '', 'code': 'api_key_is_not_allowed'}}

### 시도

In [ ]:
print(eval(filled_form))
print(type(eval(filled_form)))

{'1': ['연인', '마음', '시간'], '2': ['슬픔', '아쉬움', '혼란']}
<class 'dict'>


In [ ]:
dict_form = eval(filled_form)

for key, value in dict_form.items():
  print(key,":", value)

1 : ['연인', '마음', '시간']
2 : ['슬픔', '아쉬움', '혼란']


In [ ]:
dict_form['1']

['연인', '마음', '시간']

In [ ]:
dict_form['2']

['슬픔', '아쉬움', '혼란']

In [ ]:
chartt = chart.copy()
chartt['키워드'] = ''
chartt['감정'] = ''

chartt['키워드'][0] = dict_form['1']
chartt['감정'][0] = dict_form['2']

### 함수 정의

In [ ]:
def getting_tags(data):

  data['키워드'] = ''
  data['감정'] = ''

  keywords = []
  emotions = []

  for i in list(data.index): # 슬라이싱해서 실행해야 함 -> len에서 index로 바꿈
    my_info = data['가사'][i]

    try:
      filled_form = chain_mini.invoke(
          {"my_info": my_info,
          "form_fields": form_fields})

      # print(filled_form)
      filled_form_dic = eval(filled_form)

      keywords.append(filled_form_dic['1'])
      emotions.append(filled_form_dic['2'])

    except:
      print(f"오류 발생! index : {i}")
      keywords.append(np.nan)
      emotions.append(np.nan)

    if i % 100 == 0:
      print(f'진행상태 : {i}번째 완료')

  data['키워드'] = keywords
  data['감정'] = emotions

  return data

In [ ]:
def to_kor(data): # 한글 아닌 애들 + 길이 이상하게 나온 애들 + 오류 났던 애들 전부 한 번 더
  for i in list(data.index):

    key_list, feel_list = data['키워드'][i], data['감정'][i]
    keys = [key for key in key_list if not re.search(r'[^ㄱ-ㅎ가-힣 ]', key)] # 한글+공백인 애들만 출력
    feels = [feel for feel in feel_list if not re.search(r'[^ㄱ-ㅎ가-힣 ]', feel)] # 한글+공백인 애들만 출력

    if len(keys) != len(key_list) or len(feels) != len(feel_list) or len(key_list) > 3 or len(feel_list) > 3:

      my_info = data.loc[i,'가사']

      filled_form = chain_mini.invoke(
            {"my_info": my_info,
            "form_fields": form_fields})
      filled_form_dic = eval(filled_form)

      # print(f"{i}번째 {filled_form}")

      data['키워드'][i] = filled_form_dic['1']
      data['감정'][i] = filled_form_dic['2']

  return data

In [ ]:
def fill_na(data):

  for i in list(data[data['키워드'].isna() | data['감정'].isna()].index):
    my_info = data.loc[i,'가사'] # 오류 났던 가사 받아오기

    filled_form = chain_mini.invoke(
          {"my_info": my_info,
          "form_fields": form_fields})

    # print(filled_form) # NA 잘 사라졌는지 확인
    filled_form_dic = eval(filled_form)

    data['키워드'][i] = filled_form_dic['1']
    data['감정'][i] = filled_form_dic['2']

  return data

In [ ]:
def getting_and_correcting_tags(data1):

  try:
    data2 = fill_na(data1)
    print("NA 채우기 완료")
  except:
    print("NA 채우기 실패")
    return data1

  try:
    data3 = to_kor(data2)
    print("한글로 변환 완료")
  except:
    print("한글로 변환 실패")
    return data2

  try:
    data4 = fill_na(data3)
    print("전체 완료")
  except:
    print("마지막에서 실패")
    return data3

  return data4

### Getting Tags~~

1000곡에 17분 정도

In [ ]:
# 18부터 다시 하세요
playlist_32 = getting_tags(playlist[29000:30000])
genre_tags = playlist_30.copy() # 1000개 노래 태그

진행상태 : 29000번째 완료
오류 발생! index : 29051
진행상태 : 29100번째 완료
진행상태 : 29200번째 완료
오류 발생! index : 29215
오류 발생! index : 29282
오류 발생! index : 29294
진행상태 : 29300번째 완료
오류 발생! index : 29316
진행상태 : 29400번째 완료
오류 발생! index : 29476
진행상태 : 29500번째 완료
진행상태 : 29600번째 완료
진행상태 : 29700번째 완료
오류 발생! index : 29705
진행상태 : 29800번째 완료
진행상태 : 29900번째 완료
오류 발생! index : 29933


In [ ]:
#### 안 되면 이거 해
playlist_30 = fill_na(genre_tags)
playlist_30 = to_kor(playlist_30)
playlist_30 = fill_na(playlist_30)

TypeError: list indices must be integers or slices, not str

In [ ]:
playlist_30.to_pickle('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 7 - 가사 키워드, 해시태그 모델/가사 키워드/playlist30.pickle')

---------

In [ ]:
## 여기부터 이어서 - 마지막이다~
playlist_31 = getting_tags(playlist[30000:])
genre_tags = playlist_31.copy() # 1000개 노래 태그

진행상태 : 30000번째 완료
오류 발생! index : 30070
진행상태 : 30100번째 완료
오류 발생! index : 30104
오류 발생! index : 30113
진행상태 : 30200번째 완료
오류 발생! index : 30216
진행상태 : 30300번째 완료
오류 발생! index : 30379
오류 발생! index : 30400
진행상태 : 30400번째 완료
오류 발생! index : 30423
진행상태 : 30500번째 완료
오류 발생! index : 30532
오류 발생! index : 30546
오류 발생! index : 30578
진행상태 : 30600번째 완료
진행상태 : 30700번째 완료
오류 발생! index : 30729
진행상태 : 30800번째 완료
오류 발생! index : 30826
진행상태 : 30900번째 완료
진행상태 : 31000번째 완료
오류 발생! index : 31035
오류 발생! index : 31051
진행상태 : 31100번째 완료
진행상태 : 31200번째 완료
오류 발생! index : 31228
진행상태 : 31300번째 완료
진행상태 : 31400번째 완료
오류 발생! index : 31472
진행상태 : 31500번째 완료
진행상태 : 31600번째 완료
진행상태 : 31700번째 완료
진행상태 : 31800번째 완료
오류 발생! index : 31821
진행상태 : 31900번째 완료
진행상태 : 32000번째 완료
오류 발생! index : 32032
진행상태 : 32100번째 완료
오류 발생! index : 32154
진행상태 : 32200번째 완료
오류 발생! index : 32232
진행상태 : 32300번째 완료
오류 발생! index : 32357
진행상태 : 32400번째 완료
진행상태 : 32500번째 완료


In [ ]:
playlist_31 = fill_na(genre_tags)
playlist_31 = to_kor(playlist_31)
playlist_31 = fill_na(playlist_31)

In [ ]:
playlist_31.to_pickle('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 7 - 가사 키워드, 해시태그 모델/가사 키워드/playlist31.pickle')